# Helmholtz equation

We present a differentiable solver for the Helmholtz equation, based on `paddle-harmonics`. 

In [ ]:
import paddle
import paddle.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from math import ceil

import sys

from paddle_harmonics.sht import *
from paddle_harmonics.examples import SphereSolver

cmap='twilight_shifted'

In [ ]:
device = paddle.device.set_device('gpu' if paddle.device.cuda.device_count() > 0 else 'cpu')

We define a shallow water solver class in `shallow_water_equations.py`

In [ ]:
# initialize parameters:
nlat = 128
nlon = 2*nlat
lmax = ceil(nlon/4)
mmax = lmax

# timestepping
dt = 0.001
maxiter = int(10 / dt)

# initialize solver class
solver = SphereSolver(nlat, nlon, dt, lmax=lmax, mmax=mmax, coeff=0.01).to(device)

lons = solver.lons
lats = solver.lats

jj, ii = paddle.triu_indices(lmax, mmax)


preparing the simulation

In [ ]:
uspec0 = solver.randspec()
fig = plt.figure(figsize=(8, 6), dpi=72)
solver.plot_specdata(uspec0, fig, cmap=cmap)
plt.show()

In [ ]:
dudtspec = paddle.zeros([3, solver.lmax, solver.mmax], dtype=paddle.complex128)
nnew = 0
nnow = 1
nold = 2

uspec = uspec0.clone().to(device)
# pde = 'allen-cahn'
pde = 'ginzburg-landau'


# prepare figure for animation
fig = plt.figure(figsize=(8, 6), dpi=72)
moviewriter = animation.writers['pillow'](fps=20)
moviewriter.setup(fig, pde+'.gif', dpi=72)


with paddle.no_grad():
    for iter in range(maxiter+1):
        t = iter*dt

        if iter % 20 == 0:
            print(f"t={t:.2f}")
            plt.clf()
            solver.plot_specdata(uspec, fig, cmap=cmap)
            plt.draw()
            moviewriter.grab_frame()

        dudtspec[nnew] = solver.dudtspec(uspec, pde=pde)
        
        # forward euler, then 2nd-order adams-bashforth time steps to start.
        if iter == 0:
            dudtspec[nnow] = dudtspec[nnew]
            dudtspec[nold] = dudtspec[nnew]
        elif iter == 1:
            dudtspec[nold] = dudtspec[nnew]

        uspec = uspec + solver.dt*( (23./12.) * dudtspec[nnew] - (16./12.) * dudtspec[nnow] + (5./12.) * dudtspec[nold] )

        # implicit hyperdiffusion for vort and div.
        # uspec[1:] = swe_solver.hyperdiff * uspec[1:]
        # switch indices, do next time step.
        nsav1 = nnew
        nsav2 = nnow
        nnew = nold
        nnow = nsav1
        nold = nsav2


moviewriter.finish()